In [1]:
'''
Revised base on https://zhuanlan.zhihu.com/p/667766822
'''

# 创建DataLoader进行数据集转换
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

device = 'cpu'
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    device = 'cuda'
print(device)

class DataCreator:
    def __init__(self, batch_size, seq_len, pred_len):
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.target_slice = slice(0, None)

        self._read_data()

    # 读取数据
    def _read_data(self):
        '''
        扩展数据以缩短模型的训练时间至关重要；
        将缩放器安装在训练集上只是为了避免验证和测试集中的数据泄漏
        '''

        filepath = ('../Dataset/Simulate_Data/set_1/File_indexed.xlsx')

        df_raw = pd.read_excel(filepath)
        df = df_raw.set_index('TIME') # 以时间步为编号变量

        # split train/valid/test，以0.7/0.2/0.1的比例分割原数据集
        n = len(df)
        train_end = int(n * 0.7)
        val_end = n - int(n * 0.1)
        test_end = n

        train_df = df.iloc[:train_end]
        val_df = df.iloc[train_end - self.seq_len : val_end]
        test_df = df.iloc[val_end - self.seq_len : test_end]

        # standardize by training set
        self.scaler = StandardScaler()
        self.scaler.fit(train_df.values)

        def scale_df(df, scaler):
            data = scaler.transform(df.values)
            return pd.DataFrame(data, index=df.index, columns=df.columns)

        self.train_df = scale_df(train_df, self.scaler)
        self.val_df = scale_df(val_df, self.scaler)
        self.test_df = scale_df(test_df, self.scaler)
        self.n_feature = self.train_df.shape[-1]

    # 将数据窗口分割为输入和标签
    def _split_window(self, data):
        inputs = data[:, : self.seq_len, :]
        labels = data[:, self.seq_len :, self.target_slice]

        # inputs.set_shape([None, self.seq_len, None])
        # labels.set_shape([None, self.pred_len, None])
        return inputs, labels

    # 将数据重组为[num of samples, seq_len, input_size]的形式
    def _make_dataset(self, data, shuffle=True):
        data = np.array(data, dtype=np.float32) # 此时的data还是[total time length, input_size]的形式

        # # tensorflow 版本
        # ds = tf.keras.utils.timeseries_dataset_from_array(
        #     data=data,
        #     targets=None,
        #     sequence_length=(self.seq_len + self.pred_len),
        #     sequence_stride=1,
        #     shuffle=shuffle,
        #     batch_size=self.batch_size,
        # )
        # ds = ds.map(self._split_window)

        # pytorch 版本
        sample_size = data.shape[0] // (self.seq_len + self.pred_len)
        data = data[: sample_size * (self.seq_len + self.pred_len)]
        data = data.reshape(sample_size, self.seq_len + self.pred_len, self.n_feature)
        inputs, labels = self._split_window(data)
        inputs_tensor = torch.tensor(inputs, dtype=torch.float32).to(device)
        labels_tensor = torch.tensor(labels, dtype=torch.float32).to(device)
        # 创建一个 TensorDataset
        inputs_dataset = TensorDataset(inputs_tensor)
        labels_dataset = TensorDataset(labels_tensor)
        # 创建一个 DataLoader
        inputs_loader = DataLoader(inputs_dataset, batch_size=self.batch_size, shuffle=shuffle, generator=torch.Generator(device=device))
        labels_loader = DataLoader(labels_dataset, batch_size=self.batch_size, shuffle=shuffle, generator=torch.Generator(device=device))

        return inputs_loader, labels_loader

    # DataLoader通过对预测进行逆变换，生成训练集、验证集和测试集
    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

    def get_train(self, shuffle=True):
        # 在训练集中打乱顺序进行训练
        return self._make_dataset(self.train_df, shuffle=shuffle)

    def get_val(self):
        return self._make_dataset(self.val_df, shuffle=False)

    def get_test(self):
        return self._make_dataset(self.test_df, shuffle=False)


cuda


d:\coding\Anaconda\Software\envs\pytorch38\lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [6]:
# 构建TSMixer网络, tensorflow版本

# from tensorflow.keras import layers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def res_block(inputs, norm_type, activation, dropout, ff_dim):
    '''
    定义残差模块
    '''
    # norm = layers.BatchNormalization

    batch_norm = nn.BatchNorm1d(num_features)

    # Time mixing
    '''
    混合器层，它包括：
        批量归一化
        转置矩阵
        通过 ReLu 激活馈送到全连接层
        再次转置
        漏失层
        并在最后添加残差
    '''
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(x.shape[-1], activation='relu')(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feature mixing
    '''
    特征混合部分，其中：
        批量归一化
        致密层
        一个dropout层
        另一个致密层
        另一个dropout层
        并添加残差以进行残差连接
    '''
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation='relu')(x)  # [Batch, Input Length, FF_Dim]
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.Dropout(0.7)(x)
    return x + res

def build_model(
    input_shape,
    pred_len,
    n_block,
    ff_dim,
    target_slice,
):

    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
        x = res_block(x, norm_type, activation, dropout, ff_dim)

    if target_slice:
        x = x[:, :, target_slice]

  # Temporal projection
    '''
    时间投影步骤：
        转置
        穿过致密层
        最终转置
    '''
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

In [2]:
# 构建TSMixer网络, pytorch版本

# from tensorflow.keras import layers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TSMixer(nn.Module):
    def __init__(self, input_size, seq_len, pred_len, n_block, ff_dim, dropout_proba=0.2):
        super().__init__()
        # 定义归一化模块
        self.norm = nn.LayerNorm(input_size)
        # 定义残差块数量
        self.n_block = n_block
        # 定义致密层
        self.dense_in = nn.Linear(seq_len, seq_len)
        self.dense_out = nn.Linear(seq_len, pred_len)
        self.dense_ff_in = nn.Linear(input_size, ff_dim)
        self.dense_ff_out = nn.Linear(ff_dim, input_size)
        # 定义激活函数
        self.activate = nn.ReLU()
        # 定义dropout模块
        self.dropout = nn.Dropout(p=dropout_proba)

    def res_block(self, inputs):
        '''
        定义残差模块
        '''
        # Time mixing
        '''
        混合器层，它包括：
            批量归一化
            转置矩阵
            通过 ReLu 激活馈送到全连接层
            再次转置
            漏失层
            并在最后添加残差
        '''
        x = self.norm(inputs)
        x = x.permute(0, 2, 1)  # [Batch, Channel, Input Length]
        # print(x.size())
        x = self.dense_in(x)
        # print(x.size())
        x = self.activate(x)
        x = x.permute(0, 2, 1)  # [Batch, Input Length, Channel]
        x = self.dropout(x)
        res = x + inputs

        # Feature mixing
        '''
        特征混合部分，其中：
            批量归一化
            致密层
            一个dropout层
            另一个致密层
            另一个dropout层
            并添加残差以进行残差连接
        '''
        x = self.norm(res)
        # print(x.size())
        x = self.dense_ff_in(x)
        x = self.activate(x)
        x = self.dropout(x)
        x = self.dense_ff_out(x)
        x = self.dropout(x)
        return x + res

    def forward(self, x):
        # 输入时：[Batch, Input Length, Channel]
        for _ in range(self.n_block):
            x = self.res_block(x)

        # Temporal projection
        '''
        时间投影步骤：
            转置
            穿过致密层
            最终转置
        '''
        x = x.permute(0, 2, 1)  # [Batch, Channel, Input Length]
        x = self.dense_out(x)  # [Batch, Channel, Output Length]
        x = x.permute(0, 2, 1)  # [Batch, Output Length, Channel])

        return x

In [20]:
# 初始化DataLoader来读取数据集并创建训练集、验证集和测试集
seq_len = 8
pre_len = 16
data_loader = DataCreator(batch_size=32, seq_len=seq_len, pred_len=pre_len)

train_inputs, train_labels = data_loader.get_train()
val_inputs, val_labels = data_loader.get_val()
test_inputs, test_labels = data_loader.get_test()

# 初始化TSMixer模型
model = TSMixer(
    input_size = data_loader.n_feature,
    seq_len = seq_len,
    pred_len=pre_len,
    n_block=8,
    ff_dim=64,
    dropout_proba=0.2
)
model.to(device)

# 训练模型
'''
使用学习率为 1e-4 的 Adam 优化器;
实施检查点来保存最佳模型;
在连续三个时期没有改进的情况下提前停止以停止训练。
'''
torch.manual_seed(42)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# model.compile(optimizer, loss='mse', metrics=['mae'])

# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath='tsmixer_checkpoints/',
#     vebose=1,
#     save_best_only=True,
#     save_weights_only=True
# )

# early_stop_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',
#     patience=3
# )

# history = model.fit(
#     train_data,
#     epochs= 30,
#     validation_data=val_data,
#     callbacks=[checkpoint_callback, early_stop_callback]
# )

# 训练模型
epochs = 2000
train_loss = []
for epoch in range(epochs):
    optimizer.zero_grad()
    epoch_loss = []
    for inputs_list, label_list in zip(train_inputs, train_labels):
        # print(len(inputs_list), len(label_list))
        inputs_tensor, label_tensor = inputs_list[0].to(device), label_list[0].to(device)
        output = model(inputs_tensor)
        loss = criterion(label_tensor, output)
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, np.mean(epoch_loss)))
    train_loss.extend(epoch_loss)

model.eval()

# # 在模型训练完毕后加载检查点回调保存的最佳模型
# best_epoch = np.argmin(history.history['val_loss'])

# model.load_weights("tsmixer_checkpoints/")

# # 访问96个时间步长的最后一个窗口的预测（已按比例缩放）
# predictions = model.predict(test_inputs)

# scaled_preds = predictions[-1,:,:]

Epoch [1/2000], Loss: 5.2209
Epoch [2/2000], Loss: 5.1484
Epoch [3/2000], Loss: 4.9432
Epoch [4/2000], Loss: 4.4829
Epoch [5/2000], Loss: 4.1607
Epoch [6/2000], Loss: 4.0328
Epoch [7/2000], Loss: 3.6479
Epoch [8/2000], Loss: 3.4480
Epoch [9/2000], Loss: 3.3194
Epoch [10/2000], Loss: 3.1386
Epoch [11/2000], Loss: 2.9353
Epoch [12/2000], Loss: 2.8098
Epoch [13/2000], Loss: 2.5312
Epoch [14/2000], Loss: 2.5444
Epoch [15/2000], Loss: 2.3989
Epoch [16/2000], Loss: 2.3110
Epoch [17/2000], Loss: 2.2425
Epoch [18/2000], Loss: 2.0717
Epoch [19/2000], Loss: 2.0036
Epoch [20/2000], Loss: 1.8976
Epoch [21/2000], Loss: 1.8668
Epoch [22/2000], Loss: 1.9345
Epoch [23/2000], Loss: 1.7631
Epoch [24/2000], Loss: 1.7822
Epoch [25/2000], Loss: 1.6712
Epoch [26/2000], Loss: 1.6875
Epoch [27/2000], Loss: 1.5913
Epoch [28/2000], Loss: 1.5638
Epoch [29/2000], Loss: 1.5860
Epoch [30/2000], Loss: 1.5166
Epoch [31/2000], Loss: 1.4434
Epoch [32/2000], Loss: 1.4594
Epoch [33/2000], Loss: 1.4500
Epoch [34/2000], Lo

TSMixer(
  (norm): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
  (dense_in): Linear(in_features=8, out_features=8, bias=True)
  (dense_out): Linear(in_features=8, out_features=16, bias=True)
  (dense_ff_in): Linear(in_features=11, out_features=64, bias=True)
  (dense_ff_out): Linear(in_features=64, out_features=11, bias=True)
  (activate): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [21]:
# 使用模型进行预测
'''
对下一秒的预测和对后面第16秒的预测在数值上没有任何区别
'''
scaled_preds = []
for inputs_list, label_list in zip(test_inputs, test_labels):
    inputs_tensor, label_tensor = inputs_list[0].to(device), label_list[0].to(device)
    output = model(inputs_tensor)
    # print(output.size())
    scaled_preds.extend(output.detach().cpu().numpy())

# 将缩放和逆变换的预测存储在 DataFrame 中，以评估性能并稍后绘制预测
cols = ['ALT', 'HDOT', 'VT', 'ALPHA', 'GAMMA', 'PITCH', 'GREF', 'WXDT', 'WZ', 'VDOT', 'ALRT']

scaled_preds = np.array(scaled_preds)
scaled_preds = scaled_preds.reshape(-1, scaled_preds.shape[-1])
scaled_preds_df = pd.DataFrame(scaled_preds)
scaled_preds_df.columns = cols

preds = data_loader.inverse_transform(scaled_preds)

preds_df = pd.DataFrame(preds)
preds_df.columns = cols

# 可视化预测结果
tsmixer_preds = preds

cols_to_plot = ['ALT', 'HDOT', 'VT', 'ALPHA', 'GAMMA', 'PITCH', 'GREF', 'WXDT', 'WZ', 'VDOT', 'ALRT']

import plotly.graph_objects as go

# 交互式可视化
fig = go.Figure()
fig.add_trace(go.Scatter(y=train_loss, mode='lines', name = 'loss', line = dict(color='blue')))
fig.update_layout(
    title=f'Train loss',
    xaxis_title='Time Step',
    yaxis_title='Loss Value'
)
fig.show()

for batch in test_labels:
    label_array = batch[0].cpu().numpy()
    label_array = label_array.reshape(-1, label_array.shape[-1])
    for idx in range(len(cols_to_plot)):
        fig = go.Figure()
        # fig.add_trace(go.Scatter(y=test_X_origin[:decay_steps+pred_steps, idx], mode='lines', name = 'GT', line = dict(color='blue')))
        fig.add_trace(go.Scatter(y=label_array[:, idx], mode='lines', name = 'GT', line = dict(color='blue')))
        fig.add_trace(go.Scatter(y=scaled_preds[:, idx], mode='lines', name = 'pred_beginning', line = dict(color='red')))
        fig.update_layout(
            title=f'TSMixer Prediction vs. Ground Truth for {cols_to_plot[idx]}',
            xaxis_title='Time Step',
            yaxis_title='Value'
        )
        fig.show()


In [22]:
# 使用模型进行对全部数据进行时序预测
'''
分割比例：训练集+验证集+测试集=0.7+0.2+0.1
'''
filepath = ('../Dataset/Simulate_Data/set_1/File_indexed.xlsx')
df_raw = pd.read_excel(filepath)
df = df_raw.set_index('TIME') # 以时间步为编号变量

GT = np.array(df)
# print(GT.shape)

scaler = StandardScaler()
GT_scaled = scaler.fit_transform(GT)

pred_scaled = None
for start in range(0, GT_scaled.shape[0] - seq_len):
    input = torch.from_numpy(GT_scaled[start:start+seq_len].astype(np.float32)).to(device)
    input = input.unsqueeze(0)
    output = model(input)
    output = output.to('cpu').detach().numpy()[0]
    # print(pred_results.shape)
    if start == 0:
        pred_scaled = np.vstack((GT_scaled[:seq_len], output[0, :]))
        # print(test_X_pred.shape)
    else:
        pred_scaled = np.vstack((pred_scaled, output[0, :]))
    # break
# print(pred.shape)
        
# 以训练集的均值和方差还原为原始量纲
pred = scaler.inverse_transform(pred_scaled)

import plotly.graph_objects as go

# 交互式可视化
for idx in range(len(cols_to_plot)):
    fig = go.Figure()
    # fig.add_trace(go.Scatter(y=test_X_origin[:decay_steps+pred_steps, idx], mode='lines', name = 'GT', line = dict(color='blue')))
    fig.add_trace(go.Scatter(y=GT[:, idx], mode='lines', name = 'GT', line = dict(color='blue')))
    fig.add_trace(go.Scatter(y=pred[:, idx], mode='lines', name = 'pred_beginning', line = dict(color='red')))
    fig.update_layout(
        title=f'TSMixer Prediction vs. Ground Truth for {cols_to_plot[idx]}',
        xaxis_title='Time Step',
        yaxis_title='Value'
    )
    fig.show()